# Assignment 2: Predicting sentiment
In this assignment, you will be using the same sentiment analysis dataset as for Assignment 1, but you'll be looking to actually predict sentiment based on a variety of text-derived features.

This dataset comes from [Mass et. al. (2011)](https://www.aclweb.org/anthology/P11-1015.pdf) and the full version is available [here](http://ai.stanford.edu/~amaas/data/sentiment/).

In [8]:
# setup
import sys
import subprocess
import pkg_resources
from collections import Counter
import re
from numpy import log, mean

required = {'spacy', 'scikit-learn', 'pandas', 'transformers==2.4.1'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

import spacy
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

## Read in data
I've saved a subset of the data in the data directory on the repository.  It is available as a pickled dictionary.


In [9]:
# you will need to change this to where ever the file is stored
data_location = './data/assignment_1_reviews.pkl'
with open(data_location, 'rb') as f:
    all_text = pickle.load(f)
# corpora size
print([(k, len(all_text[k])) for k in all_text])
neg, pos = all_text.values()
# for this assignment, let's combine all our data, but maintain the labels
all_text = neg+pos
# array makes for easier indexing
is_positive = np.array([False]*len(neg)+[True]*len(pos))
# check that they're equivalent
print(np.bincount(is_positive))

[('neg', 1233), ('pos', 1266)]
[1233 1266]


## Creating document feature vectors
In this section, process all of your text data in order to create the following document-level feature vectors:

- Word Counts (using `CountVectorizer`)
- TF-IDF vectors (using `TfidfVectorizer`)
- Non-Negative Matrix Factorization-based representations (using `NMF`)
- Latent Dirichlet Allocation-based representations (using `LatentDirichletAllocation`)

All of the design elements are up to you (e.g. tokenization, vocabulary limits, number of components).  It may make sense to try out a few different designs.  In the next section we'll do some evaluation of our different strategies.

In [10]:
from spacy.lang.en import English
en = English()

In [11]:
def tokenizer(doc, model=en):
    '''Tokenizer based on example from from week_1_intro notebook.
    Filters non-alpha, url-like, and stopwords then lemmatizes each parsed token.'''
    parsed = model(doc)
    # Return list of lowercase parsed tokens that are alphanumeric and not urls 
    return([t.lemma_ for t in parsed if (t.is_alpha) and (not t.like_url) and (not t.is_stop)])

In [22]:
# Word count for all text using CountVectorizer and tokenizer
cv = CountVectorizer(tokenizer=tokenizer)
count_vectors = cv.fit_transform(all_text).toarray()
dict(zip(cv.get_feature_names(), count_vectors.sum(axis=0)))

{'-PRON-': 3,
 'aa': 1,
 'aaaaaaahhhhhhggg': 1,
 'aaaggghhhhhhh': 1,
 'aaah': 1,
 'aaam': 1,
 'aaja': 1,
 'aalcc': 1,
 'aaliyah': 4,
 'aaran': 1,
 'aardman': 2,
 'aaron': 7,
 'aawip': 2,
 'aback': 1,
 'abandon': 2,
 'abandoned': 17,
 'abandons': 1,
 'abba': 2,
 'abbey': 6,
 'abbie': 3,
 'abbot': 2,
 'abbott': 4,
 'abc': 8,
 'abdominal': 1,
 'abduction': 2,
 'abductions': 1,
 'abdul': 1,
 'abdulrahman': 1,
 'abe': 1,
 'abel': 5,
 'aberdeen': 1,
 'abernethie': 1,
 'aberration': 1,
 'aberrations': 1,
 'abhishek': 6,
 'abiding': 1,
 'abilities': 11,
 'ability': 40,
 'abit': 1,
 'able': 152,
 'ably': 4,
 'abnormal': 2,
 'aboard': 8,
 'abolished': 2,
 'abominable': 2,
 'abominably': 1,
 'abomination': 7,
 'aboriginal': 1,
 'aboriginee': 1,
 'aborted': 1,
 'abortion': 3,
 'abound': 6,
 'abounding': 1,
 'abounds': 1,
 'abraham': 3,
 'abrasive': 1,
 'abrasiveness': 1,
 'abre': 1,
 'abril': 2,
 'abroad': 7,
 'abrogated': 1,
 'abrupt': 9,
 'abruptly': 8,
 'absence': 11,
 'absent': 8,
 'absentee':

In [27]:
# Create simple tfidf vectorizer
tfidf_v = TfidfVectorizer(tokenizer=tokenizer)
# Get tfidf vectors of all text by using fit transform
tfidf_vectors = tfidf_v.fit_transform(all_text).toarray()
# Create dict from vecotrs
all_text_tfidf_dict = dict(zip(tfidf_v.get_feature_names(), tfidf_vectors.sum(axis=0)))

In [28]:
# NMF
nmf_n = 10
nmf = NMF(n_components=nmf_n)
nmf_vecs = nmf.fit_transform(tfidf_vectors)

In [30]:
# LDA
lda_n = 10
lda = LatentDirichletAllocation(n_components=lda_n)
lda_vecs = lda.fit_transform(count_vectors)

## Exploratory analysis on vectors
It's important to do some initial exploration of the features you've engineered.  Remember the goal is to get some information out of text, so you want to ensure your features are informative.  In this case, informative would mean it gives some information about sentiment.

Perform the following analysis and any additional checks that might be useful for creating a set of informative features:
- Top words for positive versus negative (Counts and TF-IDF)
- Topic model performance measures (NMF=Reconstruction error, LDA=Evidence Lower BOund (ELBO))
- Average cosine similarity between negative review vecvtors and positive review vectors (for all vectors you've created)

Tip: You can use the is_positive vector to subset your vectors.  You will likely need to have them in dense array format (use the `.toarray()` method.)

In [16]:
def get_most_frequent_words(corpus, cv=cv, num_words=10):
    '''Gets the most frequent words in a corpus, using a count vectorizer on the generated corpus dict'''
    corpus_dict = get_corpus_dict(corpus, cv)
    return sorted(corpus_dict, key=corpus_dict.get, reverse=True)[:num_words]

In [18]:
def get_corpus_dict(corpus, cv=cv):
    '''Creates a dictionary of the words and their counts in the corpus using a count vectorizer'''
    v = cv.fit_transform(corpus).toarray()
    corpus_dict = dict(zip(cv.get_feature_names(), v.sum(axis=0)))
    return corpus_dict

In [19]:
# Top 10 words - counts
# Negative reviews
neg_words_top = get_most_frequent_words(neg)
print(f'The top words in negative reviews are: {neg_words_top}')

# Positive reviews
pos_words_top = get_most_frequent_words(pos)
print(f'The top words in positive reviews are: {pos_words_top}')

The top words in negative reviews are: ['movie', 'film', 'like', 'bad', 'good', 'time', 'story', 'people', 'br', 'movies']
The top words in positive reviews are: ['film', 'movie', 'like', 'good', 'great', 'story', 'time', 'best', 'love', 'br']


In [20]:
# Top 10 words - TF-IDF
# Positive
pos_tfidf_dict = get_tfidf_dict(pos)
sorted(pos_tfidf_dict, key=pos_tfidf_dict.get, reverse=True)[:10]

['movie',
 'film',
 'good',
 'like',
 'great',
 'story',
 'time',
 'love',
 'best',
 'think']

In [21]:
# Top 10 words - TF-IDF
# Negative
neg_tfidf_dict = get_tfidf_dict(neg)
sorted(neg_tfidf_dict, key=neg_tfidf_dict.get, reverse=True)[:10]

['movie',
 'film',
 'like',
 'bad',
 'good',
 'time',
 'story',
 'br',
 'watch',
 'movies']

How do the above results look? Ideally you should see that your features give some information that might help a model discern negative from positive reviews.  That means lower similarity inter-class and different words showing up as most frequent/relevant.  Experiment with your design choices on the steps above.  Your goal should be to get to a set of vectors that have lower inter-class similarity than intra-class similarity (e.g. positive reviews should be more similar to positive reviews than negative reviews)

## Predicting sentiment
As we did in week 2's notebook, we're now going to use these informative vectors to predict sentiment.  We'll be using `LinearSVC` in this exercise, but feel free to try out other models.

Start by creating a train/test split for the dataset (typically 70%/30%).  We'll use the same split for all feature vectors for comparability. 

Do the following steps for all the feature vectors you developed above:
- Start by creating a train/test split for the dataset (typically 70%/30%).  We'll use the same split for all feature vectors for comparability. 
- Train an SVM model on your feature vectors with the corresponding target values (positive/negative)
- Test the SVM model on the test set and output the accuracy

Tip: Sklearn has a train/test split functionality for generating train/test splits (`sklearn.model_selection.train_test_split`).  Since we want to use the same reviews, make sure you set a random_state (see the docs).

Depending on how you've designed your vectors, you may find that the topic models perform worse than the count vectors.  You may want to try a couple different configurations.  

One key reason for this may be because if the goal is to use our test observations to simulate our "new observations", we haven't properly done that.  We've fit our vectorizers on the FULL corpus.  If our test observations are "unseen", that means our vectorizers should only be fit on the training corpus.

Try this out: Split the unprocessed reviews, fit the vectorizer, then the model and then transform the test observations and predict.  See how the accuracy changes

Tip: You may want to explore sklearn's `Pipelines`, which is designed for exactly this purpose